In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModelForSeq2SeqLM
from sentence_transformers import SentenceTransformer, util
from itertools import combinations
import numpy as np
import json
from tqdm import tqdm

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


# Loading Dataset : NFCorpus

In [ ]:
from datasets import load_dataset

In [ ]:
from tqdm import tqdm

In [ ]:
dataset = load_dataset("BeIR/nfcorpus", "corpus")

In [ ]:
dataset

DatasetDict({
    corpus: Dataset({
        features: ['_id', 'title', 'text'],
        num_rows: 3633
    })
})

In [ ]:
dataset_q = load_dataset("BeIR/nfcorpus", "queries")

README.md:   0%|          | 0.00/14.0k [00:00<?, ?B/s]

nfcorpus.py:   0%|          | 0.00/1.66k [00:00<?, ?B/s]

queries/queries/0000.parquet:   0%|          | 0.00/84.8k [00:00<?, ?B/s]

Generating queries split:   0%|          | 0/3237 [00:00<?, ? examples/s]

In [ ]:
dataset_q

DatasetDict({
    queries: Dataset({
        features: ['_id', 'title', 'text'],
        num_rows: 3237
    })
})

In [ ]:
dataset_qrel = load_dataset("BeIR/nfcorpus-qrels", split="test")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/14.0k [00:00<?, ?B/s]

train.tsv:   0%|          | 0.00/2.50M [00:00<?, ?B/s]

dev.tsv:   0%|          | 0.00/258k [00:00<?, ?B/s]

test.tsv:   0%|          | 0.00/280k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/110575 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11385 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/12334 [00:00<?, ? examples/s]

In [ ]:
dataset_qrel

Dataset({
    features: ['query-id', 'corpus-id', 'score'],
    num_rows: 12334
})

In [ ]:
dataset_qrel[0]

{'query-id': 'PLAIN-2', 'corpus-id': 'MED-2427', 'score': 2}

In [ ]:
from collections import defaultdict

# Initialize a defaultdict to store query-doc pairs
qrels = defaultdict(dict)

# Iterate over the dataset and populate the dictionary
for qrel in dataset_qrel:
    qrels[qrel["query-id"]][qrel["corpus-id"]] = qrel["score"]

# Convert defaultdict back to a regular dictionary if needed
qrels = dict(qrels)

In [ ]:
qrels

{'PLAIN-2': {'MED-2427': 2,
  'MED-10': 2,
  'MED-2429': 2,
  'MED-2430': 2,
  'MED-2431': 2,
  'MED-14': 2,
  'MED-2432': 2,
  'MED-2428': 1,
  'MED-2440': 1,
  'MED-2434': 1,
  'MED-2435': 1,
  'MED-2436': 1,
  'MED-2437': 1,
  'MED-2438': 1,
  'MED-2439': 1,
  'MED-3597': 1,
  'MED-3598': 1,
  'MED-3599': 1,
  'MED-4556': 1,
  'MED-4559': 1,
  'MED-4560': 1,
  'MED-4828': 1,
  'MED-4829': 1,
  'MED-4830': 1},
 'PLAIN-12': {'MED-2513': 2,
  'MED-5237': 2,
  'MED-2517': 2,
  'MED-2518': 2,
  'MED-2519': 2,
  'MED-2520': 2,
  'MED-2521': 2,
  'MED-2514': 1,
  'MED-2943': 1,
  'MED-5322': 1,
  'MED-5323': 1,
  'MED-5324': 1,
  'MED-5325': 1,
  'MED-5326': 1,
  'MED-5327': 1,
  'MED-5328': 1,
  'MED-5329': 1,
  'MED-5330': 1,
  'MED-5331': 1,
  'MED-5332': 1,
  'MED-5333': 1,
  'MED-5334': 1,
  'MED-5335': 1,
  'MED-5363': 1,
  'MED-5337': 1,
  'MED-5338': 1,
  'MED-5339': 1,
  'MED-5340': 1,
  'MED-5341': 1,
  'MED-5342': 1},
 'PLAIN-23': {'MED-2644': 2,
  'MED-2646': 2,
  'MED-2651': 2

In [ ]:
print(f"Number of query-document pairs: {len(qrels)}")

Number of query-document pairs: 323


In [ ]:
queries = {query["_id"]: query["text"] for query in dataset_q["queries"]}

In [ ]:
print(f"Number of queries: {len(queries)}")


Number of queries: 3237


In [ ]:
corpus = {doc["_id"]: doc['text'] for doc in dataset["corpus"]}

In [ ]:
print(f"Corpus size: {len(corpus)}")
print(f"Number of queries: {len(queries)}")
print(f"Number of query-document pairs: {len(qrels)}")

Corpus size: 3633
Number of queries: 3237
Number of query-document pairs: 323


In [ ]:
len(corpus)

3633

# Part 1

In [ ]:
# Load generative model (GPT-3.5 or FLAN-T5)
def load_generative_model(model_name="google/flan-t5-large"):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
    return tokenizer, model

In [ ]:
# Segment document into chunks
def segment_document(document, chunk_size, overlap):
    tokens = document.split()
    chunks = []
    for i in range(0, len(tokens), chunk_size - overlap):
        chunk = tokens[i:i + chunk_size]
        chunks.append(" ".join(chunk))
    return chunks

In [ ]:
def generate_pseudo_queries(chunks, tokenizer, model, top_k_ques=5):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)  # Move the model to the same device (GPU or CPU)

    # Prepare the prompts
    prompts = [
        f"Generate a detailed and nuanced question focusing on the most significant aspects of the passage:\n\n{chunk}\n\n"
        for chunk in chunks
    ]

    # Tokenize the input
    inputs = tokenizer(prompts, truncation=True, padding=True, return_tensors="pt", max_length=512)
    # print(inputs)


    # Move input tensors to the same device as the model
    inputs = {key: value.to(device) for key, value in inputs.items()}

    # Generate pseudo queries without gradients
    with torch.no_grad():
        outputs = model.generate(
            inputs['input_ids'],
            attention_mask=inputs['attention_mask'],
            max_length=20,
            num_return_sequences=top_k_ques,  # Number of queries to generate per chunk
            do_sample=True,  # Use sampling to generate diverse queries
            top_k=5         # Limit to the top 10 most probable next tokens
        )

    # Decode the generated sequences into queries
    queries = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    # Since `batch_decode` will return a flat list, we need to group them into batches of `top_k_ques`
    queries = [
        queries[i:i + top_k_ques]
        for i in range(0, len(queries), top_k_ques)
    ]

    return queries


In [ ]:

# Diversity filtering using semantic similarity
def filter_diverse_queries(queries, similarity_model, threshold=0.8):
    filtered_queries = []
    embeddings = similarity_model.encode(queries, convert_to_tensor=True)
    for i, query in enumerate(queries):
        is_redundant = any(
            util.cos_sim(embeddings[i], embeddings[j]) > threshold for j in range(len(filtered_queries))
        )
        if not is_redundant:
            filtered_queries.append(query)
    return filtered_queries


In [ ]:
def calculate_similarity_score(user_pseudo_query, document_pseudo_queries, similarity_model):
    """
    Calculate the similarity score for a query-document pair.

    Parameters:
    - query: The user query (string).
    - document_pseudo_queries: List of pseudo-queries for the document.
    - similarity_model: Preloaded Sentence Transformer model.

    Returns:
    - max_similarity: Maximum similarity score between query and pseudo-queries.
    """
    # Compute embeddings for the query and pseudo-queries
    # query_embedding = similarity_model.encode(user_pseudo_query, convert_to_tensor=True)
    # # pseudo_query_embeddings = similarity_model.encode(document_pseudo_queries, convert_to_tensor=True)

    # # Compute cosine similarity scores
    # similarity_scores = util.cos_sim(query_embedding, document_pseudo_queries)
    device = "cuda" if torch.cuda.is_available() else "cpu"

    similarity_model_gpu = similarity_model.to(device)

    # Compute embedding for the query on the specified device
    query_embedding = similarity_model_gpu.encode(
        user_pseudo_query,
        convert_to_tensor=True,
        device=device
    )

    if query_embedding.dim() == 1:
        query_embedding = query_embedding.unsqueeze(0)

    # Ensure pseudo-query embeddings are also on the same device
    document_pseudo_queries = document_pseudo_queries.to(device)
    # print(query_embedding.shape)
    # print(document_pseudo_queries.shape)

    # Compute cosine similarity scores
    similarity_scores = util.cos_sim(query_embedding, document_pseudo_queries)

    # Find the maximum similarity score
    max_similarity = torch.max(similarity_scores).item()
    return max_similarity



In [ ]:
def rank_documents_by_query(query, document_pseudo_queries_embeddings, similarity_model):
    """
    Rank documents based on the similarity of their pseudo-queries to the query.

    Parameters:
    - query: The user query (string).
    - document_queries_dict: Dictionary where keys are document IDs and values are lists of pseudo-queries.
    - similarity_model: Preloaded Sentence Transformer model.

    Returns:
    - ranked_scores: List of tuples (document_id, score) sorted by descending scores.
    """
    document_scores = {}
    for doc_id, pseudo_queries_embeddings in document_pseudo_queries_embeddings.items():
        # print(doc_id)
        # print(pseudo_queries)
        # print("yoo")
        score = calculate_similarity_score(query, pseudo_queries_embeddings, similarity_model)
        document_scores[doc_id] = score

    # Sort documents by scores in descending order
    ranked_scores = sorted(document_scores.items(), key=lambda x: x[1], reverse=True)
    return ranked_scores

In [ ]:
# Model names
generative_model_name = "google/flan-t5-large"
similarity_model_name = "all-mpnet-base-v2"
chunk_size=500
overlap=0
threshold=0.8

In [ ]:
similarity_model = SentenceTransformer(similarity_model_name)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# Main function to process the corpus
def process_corpus(corpus, generative_model_name, similarity_model, chunk_size, overlap, threshold):
    tokenizer, generative_model = load_generative_model(generative_model_name)

  # Initialize dictionaries to store the chunks and queries
    document_chunks = {}
    document_generated_queries = {}

    # Loop over each document in the corpus with tqdm progress bar
    for doc_id, document in tqdm(corpus.items(), desc="Processing Documents"):
        # Segment the document into chunks
        chunks = segment_document(document, chunk_size, overlap)

        # Store the chunks in a dictionary with the doc_id as the key
        document_chunks[doc_id] = chunks

        chunk_queries = []
        generated_queries = generate_pseudo_queries(chunks, tokenizer, generative_model)
        # print("Generated Queries")
        # print(generated_queries)
        # print("Generated Queries [0] ")
        # print(generated_queries[0])
        # break

        diverse_queries = filter_diverse_queries(generated_queries[0], similarity_model, threshold)
        chunk_queries.extend(diverse_queries)

        document_generated_queries[doc_id] = chunk_queries
    return document_generated_queries


# NOTE
Below is the code to call functions for generating the psuedo queries
You need not create the psuedo queries. Use the following link to find the generated psuedo queries which you can use to train the autoregressive model.

Link: https://drive.google.com/drive/folders/191D9QMsCVku2V1aCE0ZlkWvDqCzXlWQ3?usp=sharing

Check the files for their suffix to know which dataset they contain.

In [ ]:
results = process_corpus(corpus, generative_model_name, similarity_model,chunk_size, overlap, threshold)

Processing Documents: 100%|██████████| 3633/3633 [57:48<00:00,  1.05it/s]


In [ ]:
# with open("/content/drive/MyDrive/646Project/646Project/test/document_generated_queries_withdiversequeries_flan-t5-large_nfcorpus_DONOTRERUN.json", "w") as file:
#     json.dump(results, file, indent=4)
# print("Data saved as generated_queries.json")

Data saved as generated_queries.json


In [ ]:
# type(results)

## PART 2

# Part 3

Read the file from the above link and load the content from the file in data object. then use it for evaluation metrics.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

file_path = "/content/drive/MyDrive/646Project/sharedfiles/document_generated_queries_withdiversequeries_flan-t5-large_nfcorpus.json"

with open(file_path, "r") as file:
  data = json.load(file)


In [ ]:
document_pseudo_queries = data

#NOTE

For now, we used the top 100 queries from qrels to test part 3, once model is trained. The generated psuedo query for the top 100 queries in qrels should be used instead.

In [ ]:
queries_dict = list(qrels.items())[:100]

In [ ]:
type(queries_dict)

list

In [ ]:
result = [item[0] for item in queries_dict]

In [ ]:
queries_dict

[('PLAIN-2',
  {'MED-2427': 2,
   'MED-10': 2,
   'MED-2429': 2,
   'MED-2430': 2,
   'MED-2431': 2,
   'MED-14': 2,
   'MED-2432': 2,
   'MED-2428': 1,
   'MED-2440': 1,
   'MED-2434': 1,
   'MED-2435': 1,
   'MED-2436': 1,
   'MED-2437': 1,
   'MED-2438': 1,
   'MED-2439': 1,
   'MED-3597': 1,
   'MED-3598': 1,
   'MED-3599': 1,
   'MED-4556': 1,
   'MED-4559': 1,
   'MED-4560': 1,
   'MED-4828': 1,
   'MED-4829': 1,
   'MED-4830': 1}),
 ('PLAIN-12',
  {'MED-2513': 2,
   'MED-5237': 2,
   'MED-2517': 2,
   'MED-2518': 2,
   'MED-2519': 2,
   'MED-2520': 2,
   'MED-2521': 2,
   'MED-2514': 1,
   'MED-2943': 1,
   'MED-5322': 1,
   'MED-5323': 1,
   'MED-5324': 1,
   'MED-5325': 1,
   'MED-5326': 1,
   'MED-5327': 1,
   'MED-5328': 1,
   'MED-5329': 1,
   'MED-5330': 1,
   'MED-5331': 1,
   'MED-5332': 1,
   'MED-5333': 1,
   'MED-5334': 1,
   'MED-5335': 1,
   'MED-5363': 1,
   'MED-5337': 1,
   'MED-5338': 1,
   'MED-5339': 1,
   'MED-5340': 1,
   'MED-5341': 1,
   'MED-5342': 1}),
 

In [ ]:
type(result)

list

In [ ]:
# with open("result.tsv", "w") as file:
#     for item in result:
#         file.write(item + "\n")

In [ ]:

import json
with open("/content/drive/MyDrive/646Project/sharedfiles/nfcorpus_cross_encoder_results.json", "r") as file:
    cross_encoder_res = json.load(file)

result_2 = list(cross_encoder_res.keys())


In [ ]:
a_minus_b = list(set(result) - set(result_2))
b_minus_a = list(set(result_2) - set(result))

# Output the results
print("A - B:", a_minus_b)
print("B - A:", b_minus_a)

A - B: []
B - A: []


AttributeError: 'list' object has no attribute 'contains'

In [ ]:
def calculate_similarity_score(user_pseudo_query, document_pseudo_queries, similarity_model):
    """
    Calculate the similarity score for a query-document pair.

    Parameters:
    - query: The user query (string).
    - document_pseudo_queries: List of pseudo-queries for the document.
    - similarity_model: Preloaded Sentence Transformer model.

    Returns:
    - max_similarity: Maximum similarity score between query and pseudo-queries.
    """
    # Compute embeddings for the query and pseudo-queries
    # query_embedding = similarity_model.encode(user_pseudo_query, convert_to_tensor=True)
    # # pseudo_query_embeddings = similarity_model.encode(document_pseudo_queries, convert_to_tensor=True)

    # # Compute cosine similarity scores
    # similarity_scores = util.cos_sim(query_embedding, document_pseudo_queries)
    device = "cuda" if torch.cuda.is_available() else "cpu"

    similarity_model_gpu = similarity_model.to(device)

    # Compute embedding for the query on the specified device
    query_embedding = similarity_model_gpu.encode(
        user_pseudo_query,
        convert_to_tensor=True,
        device=device
    )

    if query_embedding.dim() == 1:
        query_embedding = query_embedding.unsqueeze(0)

    # Ensure pseudo-query embeddings are also on the same device
    document_pseudo_queries = document_pseudo_queries.to(device)
    # print(query_embedding.shape)
    # print(document_pseudo_queries.shape)

    # Compute cosine similarity scores
    similarity_scores = util.cos_sim(query_embedding, document_pseudo_queries)

    # Find the maximum similarity score
    max_similarity = torch.max(similarity_scores).item()
    return max_similarity



In [ ]:
def rank_documents_by_query(query, document_pseudo_queries_embeddings, similarity_model):
    """
    Rank documents based on the similarity of their pseudo-queries to the query.

    Parameters:
    - query: The user query (string).
    - document_queries_dict: Dictionary where keys are document IDs and values are lists of pseudo-queries.
    - similarity_model: Preloaded Sentence Transformer model.

    Returns:
    - ranked_scores: List of tuples (document_id, score) sorted by descending scores.
    """
    document_scores = {}
    for doc_id, pseudo_queries_embeddings in document_pseudo_queries_embeddings.items():
        # print(doc_id)
        # print(pseudo_queries)
        # print("yoo")
        score = calculate_similarity_score(query, pseudo_queries_embeddings, similarity_model)
        document_scores[doc_id] = score

    # Sort documents by scores in descending order
    ranked_scores = sorted(document_scores.items(), key=lambda x: x[1], reverse=True)
    return ranked_scores

In [ ]:
# queries_dict = list(qrels.items())[:2]
test_dict = {}
document_pseudo_queries_embeddings={}
device = "cuda" if torch.cuda.is_available() else "cpu"
similarity_model_gpu = similarity_model.to(device)

for doc_id, pseudo_queries in tqdm(document_pseudo_queries.items()):
  # print(pseudo_queries)
  # print(len(pseudo_queries))
  pseudo_query_embeddings = similarity_model_gpu.encode(pseudo_queries, convert_to_tensor=True, device = device)
  document_pseudo_queries_embeddings[doc_id] = pseudo_query_embeddings
  # # print(doc_id)
  # print(pseudo_query_embeddings[0])
  # print(len(pseudo_query_embeddings))
  # # print(document_pseudo_queries_embeddings)
  # break



100%|██████████| 3633/3633 [01:42<00:00, 35.34it/s]


In [ ]:
document_pseudo_queries

{'MED-10': ['Does a population-based study on breast cancer mortality reveal a survival advantage for statin'],
 'MED-14': ['Does statin use after breast cancer diagnosis reduce mortality due to breast cancer and all causes?',
  'Does simvastatin reduce breast cancer-specific mortality in patients with breast cancer?',
  'Is simvastatin a risk factor of breast cancer-specific mortality in '],
 'MED-118': ['Do the levels of NP and OP in nursing milk differ for the same dietary sources',
  'Which foods were associated with the consumption of 4-nonylphenol and 4-oct',
  'Are 4-Nonylphenol and 4-Octylphenol concentration in human milk'],
 'MED-301': ['Does methylmercury exposure increase susceptibility in humans and animals?',
  'How does methylmercury affect the central nervous system?',
  'Which is the primary target of MeHg in the central nervous system?',
  'What is methylmercury and what is its relation to epilepsy?'],
 'MED-306': ['Is sustained attention in the frontal lobe a particu

  # NOTE
  Replace the commented line in below code to get the generated psuedo query from the model for top 100 queries in qrels.

  *user_pseudo_query = generate_pseudo_query(queries[query_id])*


  Suggestion : Instead of generating inside the loop, it would be better to generate for all 100 queries at once and then used as needed


In [ ]:
# import json
# with open("/content/drive/MyDrive/646Project/sharedfiles/validation_result_nfcorpus.json", "r") as file:
#     user_pseudo_queries = json.load(file)

In [ ]:
# for query_id, doc_score in tqdm(queries_dict):
#   # print(query_id)
#   user_pseudo_query = generate_pseudo_query(queries[query_id], api_key)

In [ ]:
cross_encoder_res['PLAIN-186']

['Does water-dispersible hesperetin exert vasodilatory',
 'Does solitary ldl apheresis improve endothe',
 'Does postprandial triglyceride increase vasoactive vaso',
 'Does inositol hexaphosphate modulate neutrophils in',
 'Does eicosapentaenoic acid and docosahe',
 'How does cooking effect the chemoprotective and anti-proliferative properties of',
 'Does sulforaphane attenuate the oxidative stress response to',
 'Does dietary secoisolariciresinol diglucoside from fla',
 'Does inositol hexaphosphate downregulate expression of',
 'Does dietary citrate counter the acid load imparted by eucaloric high',
 'Does methionine auxotrophy have an effect on chemosensitivity?',
 'Are secoisolariciresinol and matairesinol lig',
 'Does high Urinary InsP6 Level predict Osteoporosis ',
 'Are cerebellar N-acetylaspartate/total creatine concentration',
 'Are nonylphenols estrogenic?',
 'Does methyl- cyclodextrin-derived cholesterol apopto',
 'Does salt reduction increase endothelium-dependant vasodilatio

In [ ]:
queries_dict

[('PLAIN-2',
  {'MED-2427': 2,
   'MED-10': 2,
   'MED-2429': 2,
   'MED-2430': 2,
   'MED-2431': 2,
   'MED-14': 2,
   'MED-2432': 2,
   'MED-2428': 1,
   'MED-2440': 1,
   'MED-2434': 1,
   'MED-2435': 1,
   'MED-2436': 1,
   'MED-2437': 1,
   'MED-2438': 1,
   'MED-2439': 1,
   'MED-3597': 1,
   'MED-3598': 1,
   'MED-3599': 1,
   'MED-4556': 1,
   'MED-4559': 1,
   'MED-4560': 1,
   'MED-4828': 1,
   'MED-4829': 1,
   'MED-4830': 1}),
 ('PLAIN-12',
  {'MED-2513': 2,
   'MED-5237': 2,
   'MED-2517': 2,
   'MED-2518': 2,
   'MED-2519': 2,
   'MED-2520': 2,
   'MED-2521': 2,
   'MED-2514': 1,
   'MED-2943': 1,
   'MED-5322': 1,
   'MED-5323': 1,
   'MED-5324': 1,
   'MED-5325': 1,
   'MED-5326': 1,
   'MED-5327': 1,
   'MED-5328': 1,
   'MED-5329': 1,
   'MED-5330': 1,
   'MED-5331': 1,
   'MED-5332': 1,
   'MED-5333': 1,
   'MED-5334': 1,
   'MED-5335': 1,
   'MED-5363': 1,
   'MED-5337': 1,
   'MED-5338': 1,
   'MED-5339': 1,
   'MED-5340': 1,
   'MED-5341': 1,
   'MED-5342': 1}),
 

In [ ]:
queries

{'PLAIN-3': 'Breast Cancer Cells Feed on Cholesterol',
 'PLAIN-4': 'Using Diet to Treat Asthma and Eczema',
 'PLAIN-5': 'Treating Asthma With Plants vs. Pills',
 'PLAIN-6': 'How Fruits and Vegetables Can Treat Asthma',
 'PLAIN-7': 'How Fruits and Vegetables Can Prevent Asthma',
 'PLAIN-8': 'Our Tax Dollars Subsidize Unhealthy Foods',
 'PLAIN-9': 'Reducing Arsenic in Chicken and Rice',
 'PLAIN-10': 'How Contaminated Are Our Children?',
 'PLAIN-14': 'Blood Type Diet Perceived as "Crass Fraud"',
 'PLAIN-15': 'Why Do Heart Doctors Favor Surgery and Drugs Over Diet?',
 'PLAIN-16': 'Phytates in Beans: Anti-Nutrient or Anti-Cancer?',
 'PLAIN-17': 'How Phytates Fight Cancer Cells',
 'PLAIN-18': 'Colon Cancer Prevention: Is it the Fiber or the Phytates?',
 'PLAIN-19': 'Low Carb Diets Found to Feed Heart Disease',
 'PLAIN-20': 'How Curry Can Kill Cancer Cells',
 'PLAIN-21': 'Testing Turmeric on Smokers',
 'PLAIN-24': 'Alkylphenol Endocrine Disruptors and Allergies',
 'PLAIN-25': 'Slowing Cogniti

In [ ]:
queries["PLAIN-2"]

'Do Cholesterol Statin Drugs Cause Breast Cancer?'

In [ ]:
from collections import defaultdict

# Data preprocessing to create reverse mapping
reverse_mapping = defaultdict(list)
for doc_id, pseudo_queries in document_pseudo_queries.items():
    for pseudo_query in pseudo_queries:
        reverse_mapping[pseudo_query.lower()].append(doc_id)




In [ ]:
reverse_mapping

defaultdict(list,
            {'does a population-based study on breast cancer mortality reveal a survival advantage for statin': ['MED-10'],
             'does statin use after breast cancer diagnosis reduce mortality due to breast cancer and all causes?': ['MED-14'],
             'does simvastatin reduce breast cancer-specific mortality in patients with breast cancer?': ['MED-14'],
             'is simvastatin a risk factor of breast cancer-specific mortality in ': ['MED-14'],
             'do the levels of np and op in nursing milk differ for the same dietary sources': ['MED-118'],
             'which foods were associated with the consumption of 4-nonylphenol and 4-oct': ['MED-118'],
             'are 4-nonylphenol and 4-octylphenol concentration in human milk': ['MED-118'],
             'does methylmercury exposure increase susceptibility in humans and animals?': ['MED-301'],
             'how does methylmercury affect the central nervous system?': ['MED-301'],
             'which

In [ ]:
# Initialize the result dictionary
result = {}

# Iterate over validation queries with optimized lookup
for query_id, queries in cross_encoder_res.items():
    doc_ids = []
    for query in queries:
        # Check for matches in reverse_mapping (case-insensitive)
        for pseudo_query, associated_doc_ids in reverse_mapping.items():
            if query.lower() in pseudo_query:
                doc_ids.extend(associated_doc_ids)
                break  # Stop after finding the first match
    # Remove duplicates and keep the order of document IDs
    result[query_id] = list(dict.fromkeys(doc_ids))

# Print the result
# print(result)

In [ ]:
result

{'PLAIN-2': ['MED-2559',
  'MED-3843',
  'MED-2575',
  'MED-4999',
  'MED-5330',
  'MED-2754',
  'MED-5087',
  'MED-2175',
  'MED-2426',
  'MED-4159',
  'MED-5342',
  'MED-4476',
  'MED-4987',
  'MED-2571',
  'MED-5178',
  'MED-3281',
  'MED-2644',
  'MED-3686'],
 'PLAIN-12': ['MED-3843',
  'MED-3281',
  'MED-2575',
  'MED-2559',
  'MED-3479',
  'MED-4999',
  'MED-3808',
  'MED-2754',
  'MED-3250',
  'MED-3819',
  'MED-2987',
  'MED-3178',
  'MED-1112',
  'MED-1806',
  'MED-5087',
  'MED-2426',
  'MED-5330',
  'MED-5167'],
 'PLAIN-23': ['MED-2559',
  'MED-3281',
  'MED-3843',
  'MED-2575',
  'MED-3479',
  'MED-4999',
  'MED-2754',
  'MED-5087',
  'MED-2427',
  'MED-2571',
  'MED-5178',
  'MED-5167',
  'MED-2644',
  'MED-4532',
  'MED-5330',
  'MED-3292',
  'MED-2583',
  'MED-4544',
  'MED-4951'],
 'PLAIN-33': ['MED-5330',
  'MED-3843',
  'MED-3819',
  'MED-3479',
  'MED-3250',
  'MED-2754',
  'MED-2027',
  'MED-1806',
  'MED-2069',
  'MED-5087',
  'MED-3657',
  'MED-2987',
  'MED-3281'

In [ ]:
# qrel_pseudo ={}
# for query_id, doc_score in tqdm(queries_dict):
#   # print(query_id)
#   user_pseudo_query = data[query_id]
#   qrel_pseudo[query_id] = user_pseudo_query
#   # print("Original Query:",queries[query_id])
#   # print("New Query:",user_pseudo_query)
#   # break
#   # user_pseudo_query = user_pseudo_queries[query_id]
#   if(user_pseudo_query == ""):
#     print("error")
#     break
#   doc_rank_for_query = rank_documents_by_query(user_pseudo_query, document_pseudo_queries_embeddings, similarity_model)
#   # print(type(doc_rank_for_query))
#   test_dict[query_id]={}
#   for doc_id, score in doc_rank_for_query:
#     test_dict[query_id][doc_id] = float(score)


In [ ]:
# document_pseudo_queries_embeddings

In [ ]:
test_dict = result

In [ ]:
# top_n = 10

In [ ]:
# top_documents = {}
# for query_id, doc_scores in test_dict.items():
#   # Directly take the top N documents (already sorted)
#   top_documents[query_id] = dict(list(doc_scores.items())[:top_n])

In [ ]:
# top_documents

In [ ]:
!pip install pytrec_eval

  Preparing metadata (setup.py) ... done
  Created wheel for pytrec_eval: filename=pytrec_eval-0.5-cp310-cp310-linux_x86_64.whl size=308203 sha256=bfccf22922f04483904f78ff90f05b3922a43f60347695fbedd61f117c4f28cd
  Stored in directory: /root/.cache/pip/wheels/51/3a/cd/dcc1ddfc763987d5cb237165d8ac249aa98a23ab90f67317a8
Successfully built pytrec_eval


In [ ]:
import pytrec_eval

In [ ]:
test_dict

{'PLAIN-2': ['MED-2559',
  'MED-3843',
  'MED-2575',
  'MED-4999',
  'MED-5330',
  'MED-2754',
  'MED-5087',
  'MED-2175',
  'MED-2426',
  'MED-4159',
  'MED-5342',
  'MED-4476',
  'MED-4987',
  'MED-2571',
  'MED-5178',
  'MED-3281',
  'MED-2644',
  'MED-3686'],
 'PLAIN-12': ['MED-3843',
  'MED-3281',
  'MED-2575',
  'MED-2559',
  'MED-3479',
  'MED-4999',
  'MED-3808',
  'MED-2754',
  'MED-3250',
  'MED-3819',
  'MED-2987',
  'MED-3178',
  'MED-1112',
  'MED-1806',
  'MED-5087',
  'MED-2426',
  'MED-5330',
  'MED-5167'],
 'PLAIN-23': ['MED-2559',
  'MED-3281',
  'MED-3843',
  'MED-2575',
  'MED-3479',
  'MED-4999',
  'MED-2754',
  'MED-5087',
  'MED-2427',
  'MED-2571',
  'MED-5178',
  'MED-5167',
  'MED-2644',
  'MED-4532',
  'MED-5330',
  'MED-3292',
  'MED-2583',
  'MED-4544',
  'MED-4951'],
 'PLAIN-33': ['MED-5330',
  'MED-3843',
  'MED-3819',
  'MED-3479',
  'MED-3250',
  'MED-2754',
  'MED-2027',
  'MED-1806',
  'MED-2069',
  'MED-5087',
  'MED-3657',
  'MED-2987',
  'MED-3281'

In [ ]:
test_dict = {
    query_id: {doc_id: len(docs) - rank for rank, doc_id in enumerate(docs)}
    for query_id, docs in test_dict.items()
}

In [ ]:
evaluator = pytrec_eval.RelevanceEvaluator(qrels, {'P.3,5,10', 'recall.3,5,10', 'ndcg_cut.3,5,10', 'map_cut.3,5,10'})
result = evaluator.evaluate(test_dict)
print(result)
metrics = ['P','ndcg_cut', 'recall', 'map_cut']
cutoffs = [3,5,10]
scores = {f'{metric}_{cutoff}': 0 for metric in metrics for cutoff in cutoffs}
for key in result:
  for metric in metrics:
    for cutoff in cutoffs:
      scores[f'{metric}_{cutoff}'] += result[key][f'{metric}_{cutoff}']
run_length = len(test_dict)
for score in scores:
  scores[score] /= run_length

{'PLAIN-2': {'P_3': 0.0, 'P_5': 0.0, 'P_10': 0.0, 'recall_3': 0.0, 'recall_5': 0.0, 'recall_10': 0.0, 'ndcg_cut_3': 0.0, 'ndcg_cut_5': 0.0, 'ndcg_cut_10': 0.0, 'map_cut_3': 0.0, 'map_cut_5': 0.0, 'map_cut_10': 0.0}, 'PLAIN-12': {'P_3': 0.0, 'P_5': 0.0, 'P_10': 0.0, 'recall_3': 0.0, 'recall_5': 0.0, 'recall_10': 0.0, 'ndcg_cut_3': 0.0, 'ndcg_cut_5': 0.0, 'ndcg_cut_10': 0.0, 'map_cut_3': 0.0, 'map_cut_5': 0.0, 'map_cut_10': 0.0}, 'PLAIN-23': {'P_3': 0.0, 'P_5': 0.0, 'P_10': 0.0, 'recall_3': 0.0, 'recall_5': 0.0, 'recall_10': 0.0, 'ndcg_cut_3': 0.0, 'ndcg_cut_5': 0.0, 'ndcg_cut_10': 0.0, 'map_cut_3': 0.0, 'map_cut_5': 0.0, 'map_cut_10': 0.0}, 'PLAIN-33': {'P_3': 0.3333333333333333, 'P_5': 0.2, 'P_10': 0.1, 'recall_3': 0.03125, 'recall_5': 0.03125, 'recall_10': 0.03125, 'ndcg_cut_3': 0.23463936301137822, 'ndcg_cut_5': 0.16958010263680806, 'ndcg_cut_10': 0.11366150026645364, 'map_cut_3': 0.03125, 'map_cut_5': 0.03125, 'map_cut_10': 0.03125}, 'PLAIN-44': {'P_3': 0.0, 'P_5': 0.0, 'P_10': 0.2,

In [ ]:
scores

{'P_3': 0.03,
 'P_5': 0.04000000000000001,
 'P_10': 0.03500000000000001,
 'ndcg_cut_3': 0.026485063011237066,
 'ndcg_cut_5': 0.033119532781869104,
 'ndcg_cut_10': 0.031367068497190315,
 'recall_3': 0.0023438775758787404,
 'recall_5': 0.00437015718516633,
 'recall_10': 0.00962653510259671,
 'map_cut_3': 0.001556368929711351,
 'map_cut_5': 0.0020102357230097666,
 'map_cut_10': 0.0027518163326577163}